In [1]:
from ExpectationMaximization import ExpectationMaximization
from DepthNormalization import *
import numpy as np
import matplotlib.pyplot as plt

In [2]:
kitti_path = '/scratch/local/hdd/hizkia/kitti'
dates = ['2011_09_26']
reference_rectangle = (64, 128)
min_original_rectangle = (32, 64)
funcs = [rescale_minmax, rescale_minmax_masked]
max_iters = 50

In [3]:
for func in funcs:
    print(f"Running iteration for function {func.__name__}")
    em = ExpectationMaximization(kitti_path,
                                 dates=dates,
                                 reference_rectangle=reference_rectangle,
                                 min_original_rectangle=min_original_rectangle,
                                 depth_normalization_func=func,
                                 max_iters = max_iters
                                )
    em.run()

Running iteration for function rescale_minmax
Generating samples...
Initializing EM yields a score of 87834670.74196139...
EM iteration 0 yields a score of 87834670.69687101
EM iteration 1 yields a score of 87834671.2211671
EM stopped with a score of 87834670.69687101
Running iteration for function rescale_minmax_masked
Generating samples...
Initializing EM yields a score of 86705948.0889654...
EM iteration 0 yields a score of 86720585.23953433
EM stopped with a score of 86705948.0889654


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import os
import pickle

In [ ]:
class CarDataset(Dataset):
    def __init__(self, base_path):
        self.base_path = base_path
        
    def __len__(self):
        return len(os.listdir(self.base_path))
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        with open(f'{self.base_path}/{idx}.pkl','rb') as input:
            car = pickle.load(input)
            
        return car